In [16]:
import numpy as np

import datagenerator
import warnings
import os

import seaborn as sns
from matplotlib import pyplot as plt


from sklearn.preprocessing import StandardScaler
from librosa.core.spectrum import  util

import torch
from torch import nn
from torch.nn import functional as F

from torch.utils.data import TensorDataset, DataLoader
import os

import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


import torch
from torch import nn, optim
import torch.nn.functional as F

import librosa
from joblib import dump, load

In [17]:
path = "../archive/"
set_a_timing = pd.read_csv("../archive\set_a_timing.csv")
set_a_label = pd.read_csv("../archive\set_a.csv")
set_a_timing["location"] = set_a_timing["location"]*160//441

In [28]:
batch_size = 64
mfcc_hop = 512
mfcc_frame = 2048
n_mfcc = 15
sr = 16000
n_win = 18
n_hop = 1
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cpu')

In [66]:

def frame_to_pcm(x):
    return (((x)*n_hop + n_win) * mfcc_hop + mfcc_frame)

def frame_to_sec(x):
    return frame_to_pcm(x)/sr

In [41]:
set_a_timing.head(26)

,fname,cycle,sound,location
0,set_a/normal__201102081321.wav,1,S1,3635
1,set_a/normal__201102081321.wav,1,S2,7531
2,set_a/normal__201102081321.wav,2,S1,12725
3,set_a/normal__201102081321.wav,2,S2,17140
4,set_a/normal__201102081321.wav,3,S1,22854
5,set_a/normal__201102081321.wav,3,S2,26749
6,set_a/normal__201102081321.wav,4,S1,32203
7,set_a/normal__201102081321.wav,4,S2,36878
8,set_a/normal__201102081321.wav,5,S1,41812
9,set_a/normal__201102081321.wav,5,S2,46227


In [35]:
files = set_a_timing["fname"].unique()
test = np.zeros([0,n_win,n_mfcc])
sscaler = load("std_scaler.bin")
for fname in files[:1]:
    x, _ = librosa.load(path+fname, sr=16000)
    mfccs = librosa.feature.mfcc(x, sr =sr, n_mfcc=n_mfcc, hop_length=mfcc_hop, win_length=mfcc_frame, n_mels=n_mfcc, center=False).T
    mfccs = sscaler.transform(mfccs)
    mfccs_frame = util.frame(mfccs, frame_length=n_win, hop_length=n_hop, axis =0)
    test = np.append(test,mfccs_frame, axis=0)
target_test = np.zeros(test.shape[0])
test.shape

C:\Users\slowm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\librosa\util\utils.py:204: UserWarning: librosa.util.frame called with axis=0 on a non-contiguous input. This will result in a copy.
  warnings.warn(


(226, 18, 15)

In [36]:
X_val_t =  torch.FloatTensor(test)[:,None,:,:]
y_val_t =  torch.Tensor(target_test).type(torch.LongTensor)
val_dataset = TensorDataset(X_val_t, y_val_t)
valid_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [32]:
valid_dataloader

In [37]:
import torch.nn.functional as F 

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=1)
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=3)
        self.fc1 = nn.Linear(20 * 12 * 9, 120)  # !!!
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        #print(x.shape)
        x = x.view(-1, 20 * 12 * 9)  # !!!
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [38]:
model = LeNet().to(device)
model.load_state_dict(torch.load('Net_181121.pth'))

<All keys matched successfully>

In [39]:
def evaluate(model, valid_dataloader):
    model.eval()
    prediction = np.array([])
    with torch.no_grad():
        for xb, yb in valid_dataloader:
            xb, yb = xb.to(device), yb.to(device)

            probs = model(xb)

            _, preds = torch.max(probs, axis=-1)
            prediction = np.append(prediction, preds.numpy())
    print("prediction:")
    print(np.sum([prediction[i]!=prediction[i+1] for i in range(prediction.shape[0]-1)])/2)
    print(prediction.sum())
    return prediction

In [44]:
prediction = evaluate(model, valid_dataloader)

prediction:
13.0
176.0


In [67]:
def frame_analyse(marks):
    hbflag = False
    hb_start = []
    hb_end = []
    for k,mark in enumerate(marks):
        if mark:
            if hbflag==False:
                hb_start.append(k)
            hbflag = True
        elif hbflag:
            hb_end.append(k)
            hbflag = False
    if hbflag:
            hb_end.append(k)
            hbflag = False
    return hb_start,hb_end
def func(prediction):
    starts, ends = frame_analyse(prediction)
    frame_pred = pd.DataFrame({"starts":starts,"ends":ends})
    frame_pred["duration"] = frame_pred["ends"]-frame_pred["starts"]
    frame_pred["durpcm"] = (frame_pred["duration"]-1)*mfcc_hop + mfcc_frame
    frame_pred["dursec"] = frame_pred["durpcm"]/sr
    return frame_pred

def funcb(df):
    starts = (np.array(df["location"][::2])-mfcc_frame)/mfcc_hop+1
    ends = (np.array(df["location"][1::2])-mfcc_frame)/mfcc_hop+1
    frame_pred = pd.DataFrame({"starts":starts,"ends":ends})
    frame_pred["duration"] = frame_pred["ends"]-frame_pred["starts"]
    frame_pred["durpcm"] = (frame_pred["duration"]-1)*mfcc_hop + mfcc_frame
    frame_pred["dursec"] = frame_pred["durpcm"]/sr
    return frame_pred

In [68]:
df = func(prediction)
df_true = funcb(set_a_timing[set_a_timing["fname"]=='set_a/normal__201102081321.wav'])

In [69]:
df.head()

,starts,ends,duration,durpcm,dursec
0,0,4,4,3584,0.224
1,7,22,15,9216,0.576
2,26,42,16,9728,0.608
3,45,60,15,9216,0.576
4,63,79,16,9728,0.608


In [70]:
df_true.head()

,starts,ends,duration,durpcm,dursec
0,4.099609,11.708984,7.609375,5432.0,0.339500
1,21.853516,30.476562,8.623047,5951.0,0.371937
2,41.636719,49.244141,7.607422,5431.0,0.339438
3,59.896484,69.027344,9.130859,6211.0,0.388188
4,78.664062,87.287109,8.623047,5951.0,0.371937


In [65]:
ar = np.arange(16000)
mfc = np.zeros([0,2])
frame = np.zeros([0,2])
start, end = ar[::mfcc_hop], ar[mfcc_frame::mfcc_hop]
end = end[n_win::n_hop]
start = start[:end.shape[0]]
start, end, frame_to_pcm(np.arange(10))

(array([   0,  512, 1024, 1536, 2048, 2560, 3072, 3584, 4096, 4608]),
 array([11264, 11776, 12288, 12800, 13312, 13824, 14336, 14848, 15360,
        15872]),
 array([11264, 11776, 12288, 12800, 13312, 13824, 14336, 14848, 15360,
        15872]))